# 第34章: EMアルゴリズム

## 📋 この章で学ぶこと

この章を終えると、以下ができるようになります：

- [ ] KLダイバージェンスの概念を理解できる
- [ ] ELBO（エビデンスの下界）を説明できる
- [ ] EMアルゴリズムの導出を理解できる
- [ ] GMMにEMアルゴリズムを適用できる
- [ ] EMアルゴリズムを実装できる

## 🎯 前提知識

この章を学ぶには以下の知識が必要です：

- ✅ Notebook 33（混合ガウスモデル）
- ✅ 最尤推定の基礎
- ✅ 確率の基礎（ベイズの定理、周辺化）

⏱️ **推定学習時間**: 150-180分  
📊 **難易度**: ★★★★☆（上級）  
🎓 **カテゴリ**: 実践

---

## 🌟 はじめに

前章では、GMMのパラメータ推定が「対数の中に和がある」ため解析的に解けないことを学びました。

**EMアルゴリズム（Expectation-Maximization Algorithm）**は、この問題を解決するための反復最適化アルゴリズムです。

### 🤔 EMアルゴリズムの重要性

EMアルゴリズムは、機械学習で最も重要なアルゴリズムの1つです：

1. **GMM**: クラスタリングと密度推定
2. **隠れマルコフモデル（HMM）**: 系列データの解析
3. **潜在ディリクレ配分（LDA）**: トピックモデル
4. **VAE**: EMアルゴリズムの変分近似版を使用

EMアルゴリズムを理解することは、VAEや拡散モデルの理解への重要なステップです。

In [ ]:
# ============================================================
# 環境設定
# 必要なライブラリをインポートし、グラフの表示設定を行います
# ============================================================

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.special import logsumexp
import warnings

# 警告を非表示
warnings.filterwarnings('ignore')

# グラフスタイルの設定
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# 日本語フォント設定
import matplotlib.font_manager as fm

def setup_japanese_font():
    """日本語フォントを設定する"""
    japanese_fonts = [
        'Hiragino Sans', 'Hiragino Maru Gothic Pro', 'AppleGothic',
        'Yu Gothic', 'MS Gothic',
        'Noto Sans CJK JP', 'IPAexGothic', 'TakaoPGothic',
    ]
    available_fonts = set(f.name for f in fm.fontManager.ttflist)
    for font in japanese_fonts:
        if font in available_fonts:
            plt.rcParams['font.family'] = font
            plt.rcParams['axes.unicode_minus'] = False
            return font
    return None

font_used = setup_japanese_font()
if font_used:
    print(f"日本語フォント: {font_used}")

# 再現性のための乱数シード
np.random.seed(42)

print("✅ ライブラリのインポート完了")

---

## 1. KLダイバージェンス

### 🤔 KLダイバージェンスとは？

**KLダイバージェンス（Kullback-Leibler Divergence）**は、2つの確率分布がどれだけ異なるかを測る指標です。

$$D_{KL}(q \| p) = \int q(x) \log \frac{q(x)}{p(x)} dx = \mathbb{E}_{q}\left[\log \frac{q(x)}{p(x)}\right]$$

### 📊 KLダイバージェンスの性質

1. **非負性**: $D_{KL}(q \| p) \geq 0$
2. **等号条件**: $D_{KL}(q \| p) = 0 \Leftrightarrow q = p$
3. **非対称性**: $D_{KL}(q \| p) \neq D_{KL}(p \| q)$（距離ではない！）

### 🤔 直感的な理解

KLダイバージェンスは「$q$を使って$p$を近似したときの情報損失」と解釈できます。

- $D_{KL}(q \| p)$: $p$が真の分布で、$q$で近似する場合
- 値が大きいほど、$q$による$p$の近似が悪い

In [ ]:
# ============================================================
# KLダイバージェンスの可視化
# 2つの正規分布間のKLダイバージェンスを計算・可視化します
# ============================================================

def kl_divergence_normal(mu_q, sigma_q, mu_p, sigma_p):
    """
    2つの1次元正規分布間のKLダイバージェンス
    D_KL(q || p) where q = N(mu_q, sigma_q^2), p = N(mu_p, sigma_p^2)
    
    解析解:
    D_KL(q || p) = log(sigma_p/sigma_q) + (sigma_q^2 + (mu_q - mu_p)^2) / (2*sigma_p^2) - 1/2
    """
    return (np.log(sigma_p/sigma_q) + 
            (sigma_q**2 + (mu_q - mu_p)**2) / (2*sigma_p**2) - 0.5)

# パラメータ設定
mu_p, sigma_p = 0, 1  # 真の分布 p
mu_q, sigma_q = 1, 1.5  # 近似分布 q

# KLダイバージェンスを計算
kl_q_p = kl_divergence_normal(mu_q, sigma_q, mu_p, sigma_p)
kl_p_q = kl_divergence_normal(mu_p, sigma_p, mu_q, sigma_q)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

x = np.linspace(-5, 6, 1000)
p = stats.norm.pdf(x, mu_p, sigma_p)
q = stats.norm.pdf(x, mu_q, sigma_q)

# 左: 2つの分布
axes[0].plot(x, p, 'b-', linewidth=2, label=f'p(x) = N({mu_p}, {sigma_p}²)')
axes[0].plot(x, q, 'r-', linewidth=2, label=f'q(x) = N({mu_q}, {sigma_q}²)')
axes[0].fill_between(x, p, alpha=0.3, color='blue')
axes[0].fill_between(x, q, alpha=0.3, color='red')
axes[0].set_xlabel('x', fontsize=12)
axes[0].set_ylabel('確率密度', fontsize=12)
axes[0].set_title('2つの正規分布', fontsize=14, fontweight='bold')
axes[0].legend()

# 中央: log(q/p)の可視化
log_ratio = np.log(q / p)
axes[1].plot(x, log_ratio, 'g-', linewidth=2)
axes[1].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[1].fill_between(x, log_ratio, where=log_ratio >= 0, alpha=0.3, color='green')
axes[1].fill_between(x, log_ratio, where=log_ratio < 0, alpha=0.3, color='red')
axes[1].set_xlabel('x', fontsize=12)
axes[1].set_ylabel('log(q(x)/p(x))', fontsize=12)
axes[1].set_title('log(q/p): KLダイバージェンスの被積分関数', fontsize=14, fontweight='bold')
axes[1].set_ylim(-3, 3)

# 右: KLダイバージェンスの非対称性
mu_q_range = np.linspace(-3, 3, 100)
kl_values = [kl_divergence_normal(mu, 1, 0, 1) for mu in mu_q_range]

axes[2].plot(mu_q_range, kl_values, 'purple', linewidth=2)
axes[2].axvline(x=0, color='gray', linestyle='--', alpha=0.5)
axes[2].set_xlabel('q の平均 μ_q', fontsize=12)
axes[2].set_ylabel('D_KL(q || p)', fontsize=12)
axes[2].set_title('平均の違いとKLダイバージェンス\n(σ_q = σ_p = 1)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("="*60)
print("KLダイバージェンスの計算結果")
print("="*60)
print(f"D_KL(q || p) = {kl_q_p:.4f}")
print(f"D_KL(p || q) = {kl_p_q:.4f}")
print(f"\n→ 非対称性: D_KL(q || p) ≠ D_KL(p || q)")

---

## 2. ELBO（エビデンスの下界）

### 🤔 問題の設定

潜在変数 $z$ を持つモデルで、観測データ $X$ の対数尤度を最大化したい：

$$\log p(X | \theta) = \log \int p(X, z | \theta) dz$$

この積分は一般に解析的に計算できません（GMMの場合、和になります）。

### 📊 ELBOの導出

任意の分布 $q(z)$ を導入して、対数尤度を分解します：

$$\log p(X) = \log \int p(X, z) dz = \log \int q(z) \frac{p(X, z)}{q(z)} dz$$

Jensenの不等式（$\log$ は凹関数）を適用：

$$\log p(X) \geq \int q(z) \log \frac{p(X, z)}{q(z)} dz = \mathcal{L}(q, \theta)$$

この下界 $\mathcal{L}(q, \theta)$ を**ELBO（Evidence Lower BOund）**と呼びます。

### 📊 ELBOの分解

ELBOは2通りに分解できます：

**分解1**:
$$\mathcal{L}(q, \theta) = \mathbb{E}_{q}[\log p(X, z)] - \mathbb{E}_{q}[\log q(z)]$$
$$= \mathbb{E}_{q}[\log p(X, z)] + H[q]$$

**分解2**:
$$\log p(X) = \mathcal{L}(q, \theta) + D_{KL}(q(z) \| p(z|X))$$

→ ELBOは対数尤度からKLダイバージェンスを引いたもの

In [ ]:
# ============================================================
# ELBOの可視化
# 対数尤度とELBOの関係を図示します
# ============================================================

fig, ax = plt.subplots(figsize=(10, 6))

# 概念図を描画
# 対数尤度（固定値として表示）
log_likelihood = 5

# 様々なqでのELBOとKLダイバージェンス
np.random.seed(42)
n_points = 50
kl_values = np.abs(np.random.normal(0, 2, n_points))  # KLは常に非負
elbo_values = log_likelihood - kl_values

# プロット
x_pos = np.arange(n_points)
ax.bar(x_pos, elbo_values, color='steelblue', alpha=0.7, label='ELBO')
ax.bar(x_pos, kl_values, bottom=elbo_values, color='coral', alpha=0.7, label='D_KL(q || p(z|X))')
ax.axhline(y=log_likelihood, color='green', linewidth=3, linestyle='--', label=f'log p(X) = {log_likelihood}')

ax.set_xlabel('異なる q(z) の選択', fontsize=12)
ax.set_ylabel('値', fontsize=12)
ax.set_title('ELBOと対数尤度の関係\nlog p(X) = ELBO + D_KL(q || p(z|X))', fontsize=14, fontweight='bold')
ax.legend(loc='lower right')
ax.set_ylim(-2, 7)

# 最適なqをハイライト（KL=0のとき）
best_idx = np.argmin(kl_values)
ax.bar(best_idx, elbo_values[best_idx], color='green', alpha=0.9)
ax.annotate('最適な q\n(q = p(z|X))', xy=(best_idx, log_likelihood), 
            xytext=(best_idx+5, log_likelihood+0.5),
            arrowprops=dict(arrowstyle='->', color='green'),
            fontsize=11)

plt.tight_layout()
plt.show()

print("="*60)
print("ELBOの重要な性質")
print("="*60)
print("\n1. ELBO ≤ log p(X) （常に対数尤度の下界）")
print("2. q(z) = p(z|X) のとき、ELBO = log p(X) （等号成立）")
print("3. D_KL ≥ 0 なので、ギャップは常に非負")

---

## 3. EMアルゴリズムの導出

### 📊 EMアルゴリズムのアイデア

対数尤度を直接最大化できないので、代わりにELBOを最大化します：

$$\log p(X|\theta) \geq \mathcal{L}(q, \theta)$$

EMアルゴリズムは、$q$ と $\theta$ を交互に最適化します：

1. **Eステップ**: $\theta$ を固定して、$q$ を最適化
2. **Mステップ**: $q$ を固定して、$\theta$ を最適化

### 📊 Eステップ（Expectation Step）

$\theta^{(t)}$ を固定したとき、ELBOを最大化する $q$ は：

$$q^*(z) = p(z | X, \theta^{(t)})$$

→ 現在のパラメータでの**事後分布**

このとき、$D_{KL}(q \| p(z|X)) = 0$ となり、ELBO = log p(X) となります。

### 📊 Mステップ（Maximization Step）

$q = p(z | X, \theta^{(t)})$ を固定して、$\theta$ についてELBOを最大化：

$$\theta^{(t+1)} = \arg\max_{\theta} \mathbb{E}_{q}[\log p(X, z | \theta)]$$

この期待値は「完全データの対数尤度の期待値」と呼ばれます。

In [ ]:
# ============================================================
# EMアルゴリズムの概念図
# E ステップと M ステップの関係を可視化します
# ============================================================

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# E ステップの図
ax = axes[0]
theta_fixed = 2
q_range = np.linspace(0.5, 3.5, 100)

# 疑似的なELBO曲線（qについて）
elbo_curve = -0.5 * (q_range - theta_fixed)**2 + 3
log_p = 3.5  # 対数尤度（固定）

ax.plot(q_range, elbo_curve, 'b-', linewidth=2, label='ELBO(q)')
ax.axhline(y=log_p, color='green', linestyle='--', linewidth=2, label='log p(X)')
ax.axvline(x=theta_fixed, color='red', linestyle=':', alpha=0.7)
ax.scatter([theta_fixed], [3], color='red', s=150, zorder=5, marker='*')
ax.fill_between(q_range, elbo_curve, log_p, alpha=0.2, color='coral', label='KL divergence')

ax.set_xlabel('q', fontsize=12)
ax.set_ylabel('値', fontsize=12)
ax.set_title('E ステップ: q を最適化\n(θ を固定)', fontsize=14, fontweight='bold')
ax.legend()
ax.annotate('q* = p(z|X,θ)', xy=(theta_fixed, 3), xytext=(theta_fixed+0.8, 2),
            arrowprops=dict(arrowstyle='->', color='red'), fontsize=11)

# M ステップの図
ax = axes[1]
theta_range = np.linspace(0, 4, 100)
q_fixed = 2

# 疑似的な期待対数尤度曲線（θについて）
expected_ll = -0.3 * (theta_range - 2.5)**2 + 2

ax.plot(theta_range, expected_ll, 'purple', linewidth=2, label='E_q[log p(X,z|θ)]')
ax.axvline(x=2.5, color='red', linestyle=':', alpha=0.7)
ax.scatter([2.5], [2], color='red', s=150, zorder=5, marker='*')

ax.set_xlabel('θ', fontsize=12)
ax.set_ylabel('期待対数尤度', fontsize=12)
ax.set_title('M ステップ: θ を最適化\n(q を固定)', fontsize=14, fontweight='bold')
ax.legend()
ax.annotate('θ* = argmax', xy=(2.5, 2), xytext=(3.2, 1),
            arrowprops=dict(arrowstyle='->', color='red'), fontsize=11)

# 収束の様子
ax = axes[2]
iterations = np.arange(10)
# 対数尤度が単調増加する様子をシミュレート
log_likelihood_curve = 3.5 - 2 * np.exp(-0.5 * iterations)

ax.plot(iterations, log_likelihood_curve, 'go-', linewidth=2, markersize=8, label='log p(X|θ)')
ax.axhline(y=3.5, color='gray', linestyle='--', alpha=0.5, label='収束値')
ax.set_xlabel('イテレーション', fontsize=12)
ax.set_ylabel('対数尤度', fontsize=12)
ax.set_title('EMアルゴリズムの収束\n（単調増加が保証される）', fontsize=14, fontweight='bold')
ax.legend()

plt.tight_layout()
plt.show()

print("="*60)
print("EMアルゴリズムのまとめ")
print("="*60)
print("\n【E ステップ】")
print("  現在のθで事後分布 q(z) = p(z|X,θ) を計算")
print("  → GMMでは『責任率』の計算に対応")
print("\n【M ステップ】")
print("  E_q[log p(X,z|θ)] を最大化するθを計算")
print("  → GMMでは混合係数、平均、共分散の更新に対応")

---

## 4. GMMへのEMアルゴリズムの適用

### 📊 GMMの設定

- 観測データ: $X = \{\mathbf{x}_1, ..., \mathbf{x}_N\}$
- 潜在変数: $Z = \{z_1, ..., z_N\}$、$z_n \in \{1, ..., K\}$
- パラメータ: $\theta = \{\pi_k, \boldsymbol{\mu}_k, \Sigma_k\}_{k=1}^K$

### 📊 Eステップ: 責任率の計算

$$\gamma_{nk} = p(z_n = k | \mathbf{x}_n, \theta) = \frac{\pi_k \mathcal{N}(\mathbf{x}_n | \boldsymbol{\mu}_k, \Sigma_k)}{\sum_{j=1}^{K} \pi_j \mathcal{N}(\mathbf{x}_n | \boldsymbol{\mu}_j, \Sigma_j)}$$

### 📊 Mステップ: パラメータの更新

**混合係数**:
$$\pi_k^{\text{new}} = \frac{N_k}{N}, \quad N_k = \sum_{n=1}^{N} \gamma_{nk}$$

**平均ベクトル**:
$$\boldsymbol{\mu}_k^{\text{new}} = \frac{1}{N_k} \sum_{n=1}^{N} \gamma_{nk} \mathbf{x}_n$$

**共分散行列**:
$$\Sigma_k^{\text{new}} = \frac{1}{N_k} \sum_{n=1}^{N} \gamma_{nk} (\mathbf{x}_n - \boldsymbol{\mu}_k^{\text{new}})(\mathbf{x}_n - \boldsymbol{\mu}_k^{\text{new}})^T$$

In [ ]:
# ============================================================
# GMM用EMアルゴリズムの実装
# スクラッチでEMアルゴリズムを実装します
# ============================================================

class GMMFromScratch:
    """
    EMアルゴリズムによる混合ガウスモデル（GMM）
    """
    
    def __init__(self, n_components, max_iter=100, tol=1e-6, random_state=None):
        """
        Parameters:
        -----------
        n_components : int
            混合成分の数
        max_iter : int
            最大イテレーション数
        tol : float
            収束判定の閾値
        random_state : int
            乱数シード
        """
        self.K = n_components
        self.max_iter = max_iter
        self.tol = tol
        self.random_state = random_state
        
        # パラメータ
        self.weights_ = None
        self.means_ = None
        self.covariances_ = None
        
        # 履歴
        self.log_likelihoods_ = []
        self.n_iter_ = 0
    
    def _initialize(self, X):
        """パラメータの初期化"""
        if self.random_state is not None:
            np.random.seed(self.random_state)
        
        N, d = X.shape
        
        # 混合係数: 均等に初期化
        self.weights_ = np.ones(self.K) / self.K
        
        # 平均: ランダムにデータ点を選択
        indices = np.random.choice(N, self.K, replace=False)
        self.means_ = X[indices].copy()
        
        # 共分散行列: データ全体の共分散で初期化
        self.covariances_ = np.array([np.cov(X.T) for _ in range(self.K)])
    
    def _e_step(self, X):
        """
        Eステップ: 責任率を計算
        
        Returns:
        --------
        responsibilities : array, shape (N, K)
            各データ点の各成分への責任率
        """
        N = X.shape[0]
        log_resp = np.zeros((N, self.K))
        
        for k in range(self.K):
            # 対数確率密度を計算（数値安定性のため）
            log_resp[:, k] = (np.log(self.weights_[k]) + 
                             self._log_multivariate_normal(X, self.means_[k], self.covariances_[k]))
        
        # 対数責任率を正規化（logsumexpを使用）
        log_resp_normalized = log_resp - logsumexp(log_resp, axis=1, keepdims=True)
        
        return np.exp(log_resp_normalized)
    
    def _m_step(self, X, resp):
        """
        Mステップ: パラメータを更新
        
        Parameters:
        -----------
        X : array, shape (N, d)
            データ
        resp : array, shape (N, K)
            責任率
        """
        N, d = X.shape
        
        # N_k: 各成分に割り当てられる実効サンプル数
        N_k = resp.sum(axis=0)  # shape: (K,)
        
        # 混合係数の更新
        self.weights_ = N_k / N
        
        # 平均の更新
        self.means_ = (resp.T @ X) / N_k[:, np.newaxis]  # shape: (K, d)
        
        # 共分散行列の更新
        for k in range(self.K):
            diff = X - self.means_[k]  # shape: (N, d)
            # 重み付き共分散
            weighted_cov = (resp[:, k:k+1] * diff).T @ diff / N_k[k]
            # 正則化（数値安定性のため）
            self.covariances_[k] = weighted_cov + 1e-6 * np.eye(d)
    
    def _log_multivariate_normal(self, X, mean, cov):
        """多次元正規分布の対数確率密度"""
        d = len(mean)
        diff = X - mean
        
        # コレスキー分解で安定に計算
        try:
            L = np.linalg.cholesky(cov)
            log_det = 2 * np.sum(np.log(np.diag(L)))
            y = np.linalg.solve(L, diff.T).T
            mahal = np.sum(y**2, axis=1)
        except np.linalg.LinAlgError:
            # コレスキー分解が失敗した場合
            log_det = np.log(np.linalg.det(cov) + 1e-10)
            cov_inv = np.linalg.pinv(cov)
            mahal = np.sum(diff @ cov_inv * diff, axis=1)
        
        log_pdf = -0.5 * (d * np.log(2 * np.pi) + log_det + mahal)
        return log_pdf
    
    def _compute_log_likelihood(self, X):
        """対数尤度を計算"""
        N = X.shape[0]
        log_prob = np.zeros((N, self.K))
        
        for k in range(self.K):
            log_prob[:, k] = (np.log(self.weights_[k]) + 
                             self._log_multivariate_normal(X, self.means_[k], self.covariances_[k]))
        
        return np.sum(logsumexp(log_prob, axis=1))
    
    def fit(self, X, verbose=False):
        """
        EMアルゴリズムでモデルを学習
        
        Parameters:
        -----------
        X : array, shape (N, d)
            訓練データ
        verbose : bool
            進捗を表示するか
        """
        # 初期化
        self._initialize(X)
        
        # 初期対数尤度
        prev_ll = self._compute_log_likelihood(X)
        self.log_likelihoods_ = [prev_ll]
        
        for iteration in range(self.max_iter):
            # E ステップ
            resp = self._e_step(X)
            
            # M ステップ
            self._m_step(X, resp)
            
            # 対数尤度を計算
            curr_ll = self._compute_log_likelihood(X)
            self.log_likelihoods_.append(curr_ll)
            
            if verbose and iteration % 10 == 0:
                print(f"Iteration {iteration}: log-likelihood = {curr_ll:.4f}")
            
            # 収束判定
            if abs(curr_ll - prev_ll) < self.tol:
                if verbose:
                    print(f"Converged at iteration {iteration}")
                break
            
            prev_ll = curr_ll
            self.n_iter_ = iteration + 1
        
        return self
    
    def predict(self, X):
        """各データ点を最も責任率が高い成分に割り当て"""
        resp = self._e_step(X)
        return np.argmax(resp, axis=1)
    
    def predict_proba(self, X):
        """責任率を返す"""
        return self._e_step(X)

print("✅ GMMFromScratch クラスを定義しました")

In [ ]:
# ============================================================
# EMアルゴリズムの実行とテスト
# 実装したGMMを使ってクラスタリングを行います
# ============================================================

# テストデータを生成（3成分のGMM）
np.random.seed(42)

n_samples = 500
true_weights = [0.4, 0.35, 0.25]
true_means = [
    np.array([0, 0]),
    np.array([4, 4]),
    np.array([-3, 4]),
]
true_covs = [
    np.array([[1, 0.5], [0.5, 1]]),
    np.array([[1.5, -0.5], [-0.5, 0.8]]),
    np.array([[0.8, 0], [0, 1.2]]),
]

# データ生成
samples = []
true_labels = []
for i in range(n_samples):
    k = np.random.choice(3, p=true_weights)
    x = np.random.multivariate_normal(true_means[k], true_covs[k])
    samples.append(x)
    true_labels.append(k)

X = np.array(samples)
true_labels = np.array(true_labels)

# EMアルゴリズムを実行
gmm_scratch = GMMFromScratch(n_components=3, max_iter=100, random_state=42)
gmm_scratch.fit(X, verbose=True)

# 結果を表示
print("\n" + "="*60)
print("推定結果")
print("="*60)
print(f"\n収束までのイテレーション: {gmm_scratch.n_iter_}")
print(f"\n推定された混合係数: {gmm_scratch.weights_.round(3)}")
print(f"真の混合係数: {true_weights}")

In [ ]:
# ============================================================
# EMアルゴリズムの収束過程を可視化
# 対数尤度の推移とクラスタリング結果を表示します
# ============================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# 左上: 対数尤度の推移
ax = axes[0, 0]
ax.plot(gmm_scratch.log_likelihoods_, 'bo-', linewidth=2, markersize=4)
ax.set_xlabel('イテレーション', fontsize=12)
ax.set_ylabel('対数尤度', fontsize=12)
ax.set_title('EMアルゴリズムの収束\n（対数尤度は単調増加）', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

# 右上: 真のラベル
ax = axes[0, 1]
colors = ['steelblue', 'coral', 'green']
for k in range(3):
    mask = true_labels == k
    ax.scatter(X[mask, 0], X[mask, 1], c=colors[k], alpha=0.6, s=30, label=f'成分 {k+1}')
    ax.scatter(true_means[k][0], true_means[k][1], c=colors[k], s=300, 
              marker='*', edgecolors='black', linewidths=2)
ax.set_xlabel('X₁', fontsize=12)
ax.set_ylabel('X₂', fontsize=12)
ax.set_title('真のラベル\n（★ = 真の平均）', fontsize=14, fontweight='bold')
ax.legend()
ax.set_aspect('equal')

# 左下: 推定されたラベル
ax = axes[1, 0]
predicted_labels = gmm_scratch.predict(X)
for k in range(3):
    mask = predicted_labels == k
    ax.scatter(X[mask, 0], X[mask, 1], c=colors[k], alpha=0.6, s=30, label=f'成分 {k+1}')
    ax.scatter(gmm_scratch.means_[k, 0], gmm_scratch.means_[k, 1], c=colors[k], s=300, 
              marker='X', edgecolors='black', linewidths=2)
ax.set_xlabel('X₁', fontsize=12)
ax.set_ylabel('X₂', fontsize=12)
ax.set_title('EMアルゴリズムの結果\n（X = 推定された平均）', fontsize=14, fontweight='bold')
ax.legend()
ax.set_aspect('equal')

# 右下: 確率密度の等高線
ax = axes[1, 1]
x = np.linspace(-6, 8, 200)
y = np.linspace(-3, 8, 200)
X_grid, Y_grid = np.meshgrid(x, y)
XY = np.column_stack([X_grid.ravel(), Y_grid.ravel()])

# 推定されたGMMの密度を計算
Z = np.zeros(XY.shape[0])
for k in range(3):
    rv = stats.multivariate_normal(gmm_scratch.means_[k], gmm_scratch.covariances_[k])
    Z += gmm_scratch.weights_[k] * rv.pdf(XY)
Z = Z.reshape(X_grid.shape)

ax.contourf(X_grid, Y_grid, Z, levels=20, cmap='viridis', alpha=0.7)
ax.contour(X_grid, Y_grid, Z, levels=10, colors='white', alpha=0.5)
ax.scatter(X[:, 0], X[:, 1], c='white', alpha=0.2, s=10)
for k in range(3):
    ax.scatter(gmm_scratch.means_[k, 0], gmm_scratch.means_[k, 1], c='red', s=200, 
              marker='*', edgecolors='white', linewidths=2)
ax.set_xlabel('X₁', fontsize=12)
ax.set_ylabel('X₂', fontsize=12)
ax.set_title('推定されたGMM密度', fontsize=14, fontweight='bold')
ax.set_aspect('equal')

plt.tight_layout()
plt.show()

---

## 5. EMアルゴリズムの収束性質

### 📊 収束の保証

EMアルゴリズムには以下の重要な性質があります：

1. **単調増加性**: 各イテレーションで対数尤度は減少しない
   $$\log p(X | \theta^{(t+1)}) \geq \log p(X | \theta^{(t)})$$

2. **収束の保証**: 局所最適解に収束する（ただし大域最適とは限らない）

3. **初期値依存性**: 結果は初期値に依存する

In [ ]:
# ============================================================
# 初期値の影響
# 異なる初期値での収束を比較します
# ============================================================

fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# 6つの異なる初期値で実験
seeds = [42, 123, 456, 789, 101, 202]

for idx, seed in enumerate(seeds):
    ax = axes[idx // 3, idx % 3]
    
    # EMアルゴリズムを実行
    gmm = GMMFromScratch(n_components=3, max_iter=100, random_state=seed)
    gmm.fit(X)
    
    # 対数尤度の推移をプロット
    ax.plot(gmm.log_likelihoods_, 'o-', linewidth=2, markersize=3)
    ax.set_xlabel('イテレーション', fontsize=11)
    ax.set_ylabel('対数尤度', fontsize=11)
    final_ll = gmm.log_likelihoods_[-1]
    ax.set_title(f'初期値 seed={seed}\n最終対数尤度: {final_ll:.2f}', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)

plt.suptitle('初期値によるEMアルゴリズムの収束の違い', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("💡 ポイント:")
print("  - 初期値によって収束先（局所最適解）が異なることがある")
print("  - 実務では複数の初期値で実行し、最良の結果を選ぶ")
print("  - sklearn の GaussianMixture は n_init パラメータでこれを行う")

In [ ]:
# ============================================================
# EMアルゴリズムのステップごとの可視化
# 各イテレーションでのクラスタリング結果を表示します
# ============================================================

# 新しいGMMを作成（途中経過を保存するため）
class GMMWithHistory(GMMFromScratch):
    def fit_with_history(self, X):
        """各イテレーションの状態を保存しながらフィット"""
        self._initialize(X)
        
        self.history_ = {
            'means': [self.means_.copy()],
            'labels': [self.predict(X)],
            'log_likelihood': [self._compute_log_likelihood(X)]
        }
        
        for iteration in range(self.max_iter):
            resp = self._e_step(X)
            self._m_step(X, resp)
            
            self.history_['means'].append(self.means_.copy())
            self.history_['labels'].append(self.predict(X))
            self.history_['log_likelihood'].append(self._compute_log_likelihood(X))
            
            if iteration > 0:
                if abs(self.history_['log_likelihood'][-1] - self.history_['log_likelihood'][-2]) < self.tol:
                    break
        
        return self

# 実行
np.random.seed(42)
gmm_history = GMMWithHistory(n_components=3, max_iter=50, random_state=123)
gmm_history.fit_with_history(X)

# イテレーション 0, 1, 2, 5, 10, 最終 を表示
iterations_to_show = [0, 1, 2, 5, 10, len(gmm_history.history_['means'])-1]

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for ax_idx, iter_idx in enumerate(iterations_to_show):
    ax = axes[ax_idx]
    
    labels = gmm_history.history_['labels'][iter_idx]
    means = gmm_history.history_['means'][iter_idx]
    ll = gmm_history.history_['log_likelihood'][iter_idx]
    
    for k in range(3):
        mask = labels == k
        ax.scatter(X[mask, 0], X[mask, 1], c=colors[k], alpha=0.5, s=20)
        ax.scatter(means[k, 0], means[k, 1], c=colors[k], s=200, 
                  marker='*', edgecolors='black', linewidths=2)
    
    ax.set_xlabel('X₁', fontsize=11)
    ax.set_ylabel('X₂', fontsize=11)
    ax.set_title(f'Iteration {iter_idx}\nlog L = {ll:.2f}', fontsize=12, fontweight='bold')
    ax.set_aspect('equal')
    ax.set_xlim(-6, 8)
    ax.set_ylim(-3, 8)

plt.suptitle('EMアルゴリズムの収束過程', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---

## 6. EMアルゴリズムとVAEの関係

### 📊 EMとVAEの対応

VAE（変分オートエンコーダ）は、EMアルゴリズムの考え方を拡張したものです：

| 概念 | GMM + EM | VAE |
|------|----------|-----|
| 潜在変数 | 離散（どの成分か） | 連続（潜在ベクトル） |
| デコーダ | $p(x|z) = \mathcal{N}(\mu_k, \Sigma_k)$ | ニューラルネット |
| 事後分布 | $p(z|x)$ を解析的に計算 | エンコーダで近似 |
| 最適化 | ELBOを交互最適化 | ELBOを勾配降下で最適化 |

VAEの損失関数はまさにELBOの負値です：

$$\mathcal{L}_{VAE} = -\mathcal{L}_{ELBO} = -\mathbb{E}_{q}[\log p(x|z)] + D_{KL}(q(z|x) \| p(z))$$

→ 再構成誤差 + KL正則化項

In [ ]:
# ============================================================
# EMとVAEの関係を図解
# 概念的な対応を可視化します
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左: GMMの潜在変数（離散）
ax = axes[0]
ax.text(0.5, 0.95, 'GMM + EMアルゴリズム', ha='center', fontsize=14, fontweight='bold', transform=ax.transAxes)

# 潜在変数（離散）
for i, (x, label) in enumerate(zip([0.2, 0.5, 0.8], ['z=1', 'z=2', 'z=3'])):
    circle = plt.Circle((x, 0.7), 0.08, color=colors[i], alpha=0.7)
    ax.add_patch(circle)
    ax.text(x, 0.7, label, ha='center', va='center', fontsize=11, fontweight='bold')

ax.text(0.5, 0.85, '潜在変数 z（離散）', ha='center', fontsize=11)

# 矢印
for x in [0.2, 0.5, 0.8]:
    ax.annotate('', xy=(x, 0.45), xytext=(x, 0.6),
                arrowprops=dict(arrowstyle='->', color='black', lw=2))

# デコーダ
for i, x in enumerate([0.2, 0.5, 0.8]):
    ax.text(x, 0.35, f'N(μ{i+1}, Σ{i+1})', ha='center', fontsize=10,
           bbox=dict(boxstyle='round', facecolor=colors[i], alpha=0.3))

# 観測データ
ax.text(0.5, 0.15, 'x（観測データ）', ha='center', fontsize=12,
       bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.5))

for x in [0.2, 0.5, 0.8]:
    ax.annotate('', xy=(0.5, 0.2), xytext=(x, 0.3),
                arrowprops=dict(arrowstyle='->', color='gray', lw=1))

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')

# 右: VAEの潜在変数（連続）
ax = axes[1]
ax.text(0.5, 0.95, 'VAE', ha='center', fontsize=14, fontweight='bold', transform=ax.transAxes)

# 潜在空間（連続）
theta = np.linspace(0, 2*np.pi, 100)
r = 0.15
ax.fill(0.5 + r*np.cos(theta), 0.7 + r*0.7*np.sin(theta), alpha=0.3, color='purple')
ax.text(0.5, 0.7, 'z ∈ ℝ^d', ha='center', va='center', fontsize=11, fontweight='bold')
ax.text(0.5, 0.88, '潜在変数 z（連続）', ha='center', fontsize=11)

# エンコーダ
ax.annotate('', xy=(0.3, 0.65), xytext=(0.3, 0.25),
            arrowprops=dict(arrowstyle='->', color='blue', lw=2))
ax.text(0.15, 0.45, 'Encoder\nq(z|x)', ha='center', fontsize=10, color='blue')

# デコーダ
ax.annotate('', xy=(0.7, 0.25), xytext=(0.7, 0.55),
            arrowprops=dict(arrowstyle='->', color='red', lw=2))
ax.text(0.85, 0.45, 'Decoder\np(x|z)', ha='center', fontsize=10, color='red')

# 観測データ
ax.text(0.5, 0.15, 'x（観測データ）', ha='center', fontsize=12,
       bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.5))

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')

plt.tight_layout()
plt.show()

print("="*60)
print("EMアルゴリズムからVAEへ")
print("="*60)
print("\n1. 潜在変数を離散から連続に拡張")
print("2. デコーダをニューラルネットワークに")
print("3. 事後分布をエンコーダで近似")
print("4. ELBOを勾配降下法で最適化")
print("\n→ これがVAEの基本的なアイデアです！")

---

## まとめ

### 🎯 このノートブックで学んだこと

**KLダイバージェンス**
- ✓ 2つの分布の「違い」を測る指標
- ✓ 非負性、非対称性を持つ
- ✓ $D_{KL}(q \| p) = \mathbb{E}_q[\log(q/p)]$

**ELBO**
- ✓ 対数尤度の下界: $\log p(X) \geq \mathcal{L}(q, \theta)$
- ✓ $\log p(X) = \mathcal{L} + D_{KL}(q \| p(z|X))$
- ✓ $q = p(z|X)$ のとき等号成立

**EMアルゴリズム**
- ✓ E ステップ: $q(z) = p(z|X, \theta^{(t)})$ を計算（責任率）
- ✓ M ステップ: $\theta$ を更新（混合係数、平均、共分散）
- ✓ 対数尤度は単調増加、局所最適解に収束

### 📊 GMMのEMアルゴリズム チートシート

| ステップ | 計算内容 |
|---------|----------|
| **E ステップ** | $\gamma_{nk} = \frac{\pi_k \mathcal{N}(x_n \| \mu_k, \Sigma_k)}{\sum_j \pi_j \mathcal{N}(x_n \| \mu_j, \Sigma_j)}$ |
| **M ステップ (π)** | $\pi_k = N_k / N$ |
| **M ステップ (μ)** | $\mu_k = \frac{1}{N_k}\sum_n \gamma_{nk} x_n$ |
| **M ステップ (Σ)** | $\Sigma_k = \frac{1}{N_k}\sum_n \gamma_{nk}(x_n - \mu_k)(x_n - \mu_k)^T$ |

### 🔧 実装のポイント

```python
# 数値安定性のために対数空間で計算
from scipy.special import logsumexp

# E ステップ
log_resp = log_weights + log_pdf  # 対数責任率（非正規化）
log_resp = log_resp - logsumexp(log_resp, axis=1, keepdims=True)  # 正規化
resp = np.exp(log_resp)
```

---

### ⚠️ よくあるエラー

#### エラー #1: 共分散行列の特異性

```python
# ❌ 問題: 共分散行列が特異になる
# 1つの成分にデータが集中すると発生

# ✅ 解決策: 正則化を追加
covariance = weighted_cov + 1e-6 * np.eye(d)
```

#### エラー #2: アンダーフロー

```python
# ❌ 問題: 確率密度が非常に小さくなる
resp = weights * pdf  # アンダーフローの可能性

# ✅ 解決策: 対数空間で計算
log_resp = np.log(weights) + log_pdf
log_resp = log_resp - logsumexp(log_resp, axis=1, keepdims=True)
resp = np.exp(log_resp)
```

---

## 🎓 自己評価クイズ

### Q1: EMアルゴリズムが対数尤度を直接最大化せず、ELBOを最大化する理由は？

<details>
<summary>💡 答えを見る</summary>

**答え**: 対数尤度に潜在変数についての和（または積分）が含まれるため、解析的に最大化できないから

$$\log p(X) = \log \sum_z p(X, z)$$

対数の中に和があると、微分しても閉形式の解が得られません。ELBOを使うことで、この問題を回避できます。

</details>

---

### Q2: EステップとMステップ、それぞれ何を最適化していますか？

<details>
<summary>💡 答えを見る</summary>

**答え**:

- **E ステップ**: $\theta$ を固定して、$q(z)$ を最適化
  - 最適解: $q^*(z) = p(z|X, \theta)$（事後分布）
  - GMMでは責任率の計算に対応

- **M ステップ**: $q(z)$ を固定して、$\theta$ を最適化
  - 完全データの期待対数尤度を最大化
  - GMMでは混合係数、平均、共分散の更新に対応

</details>

---

### Q3: EMアルゴリズムは必ず大域最適解に収束しますか？

<details>
<summary>💡 答えを見る</summary>

**答え**: いいえ、局所最適解に収束する可能性があります

EMアルゴリズムは対数尤度を単調に増加させますが、収束先は初期値に依存します。そのため：

1. 複数の初期値で実行する（`n_init`パラメータ）
2. 最も対数尤度が高い結果を選ぶ

という方法が一般的です。

</details>

---

### Q4: VAEの損失関数とELBOの関係は？

<details>
<summary>💡 答えを見る</summary>

**答え**: VAEの損失関数 = -ELBO

$$\mathcal{L}_{VAE} = -\mathbb{E}_{q(z|x)}[\log p(x|z)] + D_{KL}(q(z|x) \| p(z))$$

- 第1項: 再構成誤差（デコーダの性能）
- 第2項: KL正則化項（潜在空間の正則化）

これを最小化することは、ELBOを最大化することと同じです。

</details>

---

### 📚 練習問題

1. **1次元GMM**: 1次元のGMM（2成分）に対するEMアルゴリズムを実装し、身長データ（男女混合）でテストしてください。

2. **収束速度の比較**: 異なる初期化方法（ランダム vs K-means++）での収束速度を比較してください。

3. **成分数の推定**: BICを使って最適な成分数を自動的に選択する機能を追加してください。

---

### ✅ 学習チェックリスト

- [ ] KLダイバージェンスの定義と性質を説明できる
- [ ] ELBOと対数尤度の関係を説明できる
- [ ] EMアルゴリズムのE・Mステップを説明できる
- [ ] GMMのEMアルゴリズムを実装できる
- [ ] EMアルゴリズムとVAEの関係を理解している

---

**次のステップ**: Notebook 35で、**PyTorch基礎と勾配法**を学びます！

ここからはPyTorchを使ったディープラーニングの実装に進みます。VAEや拡散モデルを実装するための基礎を固めましょう。